DATA PREPARATION

In [83]:
#I decided to use beautiful soup library to gather data from Wikipedia

# Beautifulsoup installation 
!conda install -c anaconda beautifulsoup4 --yes

print('done')

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.8.1

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs: 
    - beautifulsoup4


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2019.11.28         |           py36_0         156 KB  anaconda

The following packages will be UPDATED:

    certifi: 2019.11.28-py36_0 conda-forge --> 2019.11.28-py36_0 anaconda
    openssl: 1.1.1d-h516909a_0 conda-forge --> 1.1.1-h7b6447c_0  anaconda


certifi-2019.11.28   | 156 KB    | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction: done
done


In [84]:
#updating
#conda update -n base -c deafaults conda 
#print('update complete')

In [85]:
#lxml install
!conda install lxml --yes

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.8.1

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs: 
    - lxml


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2019.11.28         |           py36_0         156 KB

The following packages will be UPDATED:

    certifi: 2019.11.28-py36_0 anaconda --> 2019.11.28-py36_0

The following packages will be DOWNGRADED:

    openssl: 1.1.1-h7b6447c_0  anaconda --> 1.1.1d-h7b6447c_3


certifi-2019.11.28   | 156 KB    | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction: done


In [86]:
# html5lib install
!conda install html5lib

Solving environment: \ 
  - conda-forge::html5lib-0.9999999-py36_0
  - defaults::html5lib-0.9999999-py36done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.8.1

Please update conda by running

    $ conda update -n base -c defaults conda



# All requested packages already installed.



In [87]:
# Requests installation to open URL
!conda install requests --yes

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.8.1

Please update conda by running

    $ conda update -n base -c defaults conda



# All requested packages already installed.



In [88]:
#Importing library
from bs4 import BeautifulSoup
import requests
print('library imported')
print('\n\n')


library imported





DATA PREPARATION USING PANDA

In [89]:
import pandas as pd
import numpy as np

In [90]:
# Read table from wiki page
data_list = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M', header = 0)

# to show the table and verify if it is correct
table = data_list[0]

# creating a dataframe
data_df = pd.DataFrame(table)

First operation

In [91]:
#Cleaning the table by ignoring cells with a borough that is Not assigned.
filter1 = data_df.drop(data_df[data_df['Borough'] == 'Not assigned'].index, axis=0)
print('shape after first filter:', filter1.shape)

shape after first filter: (210, 3)


Second Operation

In [92]:
#Clean data by assigning the same borough when a neighbourhood is not assigned
#Looking for indexes where neighbourhoods are not assigned
na_index = filter1.index[filter1['Neighbourhood'] == 'Not assigned'].tolist()

# Considering that we have only one single not assigned value, I assign Borough name to Neighbourhood by hand
filter1['Neighbourhood'] = filter1['Neighbourhood'].replace('Not assigned', 'Queen\'s Park')

Third Operation

In [93]:
#Combining rows with the same postal code and different neighbourhood by using function groupby
    # grouped by Postcode, then it will be renamed in Postalcode
    # Neighbourhoods belonging to the same postal code will be added together separated by a comma using .agg and.join function
    # Index is reset
    
groupby1 =filter1.groupby('Postcode').agg({'Borough' : 'first', 'Neighbourhood' : ','.join}).reset_index().reindex(columns=filter1.columns)

print('After groupby function we have a dataframe:\n', type(groupby1))

# Column name assignment
groupby1.columns = ['PostalCode', 'Borough','Neighborhood']

After groupby function we have a dataframe:
 <class 'pandas.core.frame.DataFrame'>


Final dimension check --> Shape function

In [94]:
# Checking dataframe dimensions
print('\n\nfinal dataframe dimensions:', groupby1.shape)

#Printing dataframe
groupby1.head(10)



final dataframe dimensions: (103, 3)


,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park"
7,M1L,Scarborough,"Clairlea,Golden Mile,Oakridge"
8,M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff,Cliffside West"


PART 2

In [95]:
# Importing Geospatial Data
lat_long = pd.read_csv('http://cocl.us/Geospatial_data')
lat_long.head(10)

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
5,M1J,43.744734,-79.239476
6,M1K,43.727929,-79.262029
7,M1L,43.711112,-79.284577
8,M1M,43.716316,-79.239476
9,M1N,43.692657,-79.264848


In [96]:
#Checking geospatial dimensions
lat_long.shape # Dimensions are the same as the final dataframe with postalcodes

(103, 3)

In [97]:
#sorting previous dataframe in ascending order with respect to Postalcode column
groupby1.sort_values(by=['PostalCode'])

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
...,...,...,...
98,M9N,York,Weston
99,M9P,Etobicoke,Westmount
100,M9R,Etobicoke,"Kingsview Village,Martin Grove Gardens,Richvie..."
101,M9V,Etobicoke,"Albion Gardens,Beaumond Heights,Humbergate,Jam..."


In [98]:
# Dropping Postalcode to avoid duplicate
dropped_lat_long=lat_long.drop(columns='Postal Code')
#dropped_lat_long

In [99]:
#merging geospadial data with postalcode dataframe. 
final_df = pd.concat([groupby1,dropped_lat_long], axis=1)

GEOSPATIAL DATAFRAME OF TORONTO IN ALPHABETICAL ORDER IN POSTALCODE

In [100]:
final_df.head(20)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea,Golden Mile,Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff,Cliffside West",43.692657,-79.264848


PART 3 - The procedure followed here is the same as previous lab with clusterin in NY

In [101]:
!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.8.1

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2019.11.28         |           py36_0         149 KB  conda-forge

The following packages will be UPDATED:

    certifi: 2019.11.28-py36_0 --> 2019.11.28-py36_0 conda-forge

The following packages will be DOWNGRADED:

    openssl: 1.1.1d-h7b6447c_3 --> 1.1.1d-h516909a_0 conda-forge


certifi-2019.11.28   | 149 KB    | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction: done


In [102]:
!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.8.1

Please update conda by running

    $ conda update -n base -c defaults conda



# All requested packages already installed.



In [103]:
import json # library to handle JSON files
from pandas.io.json import json_normalize

Use geopy library to get the latitude and longitude values of Toronto

In [104]:
address = 'Toronto, TO'

geolocator = Nominatim(user_agent="to_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of toronto are 43.653963, -79.387207.


Create a map of New York with neighborhoods superimposed on top

In [105]:
# create map of New York using latitude and longitude values
map_to = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(final_df['Latitude'], final_df['Longitude'], final_df['Borough'], final_df['Neighborhood']):
    label = '{}, {}'.format(final_df, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_to)

    

In [106]:
# Showing the map 
map_to

In [107]:
# To simplify, let's consider only borough called Scarborough

toro_data = final_df[final_df['Borough'] == 'Scarborough'].reset_index(drop=True)
toro_data.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


Define Foursquare Credentials and Version

In [108]:
CLIENT_ID = 'SG3N4MO5HDS5VD2YQ0YH0AQ0PCIQZODCQZBD14DYFHWTYLUD' # your Foursquare ID
CLIENT_SECRET = 'NKQPT5LTHE0DVYLZ4IM1PWSLX5XUMM42CNQXFGX5GOG12KIY' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: SG3N4MO5HDS5VD2YQ0YH0AQ0PCIQZODCQZBD14DYFHWTYLUD
CLIENT_SECRET:NKQPT5LTHE0DVYLZ4IM1PWSLX5XUMM42CNQXFGX5GOG12KIY


In [109]:
toro_data.loc[0, 'Neighborhood']

neighborhood_latitude = toro_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = toro_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = toro_data.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Rouge,Malvern are 43.806686299999996, -79.19435340000001.


In [110]:
# type your answer here
radius = 800
LIMIT = 15 #limit of numbers of venues, as requested

In [111]:
#Double-click __here__ for the solution.

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)


In [112]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5e33c754b1cac0001b2145dc'},
 'response': {'headerLocation': 'Malvern',
  'headerFullLocation': 'Malvern, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 15,
  'suggestedBounds': {'ne': {'lat': 43.8138863072, 'lng': -79.18439528916866},
   'sw': {'lat': 43.79948629279999, 'lng': -79.20431151083136}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4d669cba83865481c948fa53',
       'name': 'Images Salon & Spa',
       'location': {'address': '8130 Sheppard Ave E',
        'crossStreet': 'Morningside Ave',
        'lat': 43.80228301948931,
        'lng': -79.19856472801668,
        'labeledLatLngs': [{'label': 'display',
          'lat': 43.80228301948931,
          'lng': -79.19856472801668}],
        'distanc

In [113]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [114]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Images Salon & Spa,Spa,43.802283,-79.198565
1,Wendy's,Fast Food Restaurant,43.802008,-79.198080
2,Wendy's,Fast Food Restaurant,43.807448,-79.199056
3,Staples Morningside,Paper / Office Supplies Store,43.800285,-79.196607
4,Tim Hortons,Coffee Shop,43.802000,-79.198169


In [115]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

15 venues were returned by Foursquare.


Explore Neighborhoods

In [116]:
#Let's create a function to repeat the same process to all the neighborhoods

def getNearbyVenues(names, latitudes, longitudes, radius=1000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [117]:
to_venues = getNearbyVenues(names=toro_data['Neighborhood'],
                                   latitudes=toro_data['Latitude'],
                                   longitudes=toro_data['Longitude']
                                  )


Rouge,Malvern
Highland Creek,Rouge Hill,Port Union
Guildwood,Morningside,West Hill
Woburn
Cedarbrae
Scarborough Village
East Birchmount Park,Ionview,Kennedy Park
Clairlea,Golden Mile,Oakridge
Cliffcrest,Cliffside,Scarborough Village West
Birch Cliff,Cliffside West
Dorset Park,Scarborough Town Centre,Wexford Heights
Maryvale,Wexford
Agincourt
Clarks Corners,Sullivan,Tam O'Shanter
Agincourt North,L'Amoreaux East,Milliken,Steeles East
L'Amoreaux West
Upper Rouge


In [118]:
print(to_venues.shape)

(216, 7)


Let's check how many venues were returned for each neighborhood

In [119]:
to_venues.groupby('Neighborhood').count()


,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agincourt,15,15,15,15,15,15
"Agincourt North,L'Amoreaux East,Milliken,Steeles East",15,15,15,15,15,15
"Birch Cliff,Cliffside West",13,13,13,13,13,13
Cedarbrae,15,15,15,15,15,15
"Clairlea,Golden Mile,Oakridge",15,15,15,15,15,15
"Clarks Corners,Sullivan,Tam O'Shanter",15,15,15,15,15,15
"Cliffcrest,Cliffside,Scarborough Village West",12,12,12,12,12,12
"Dorset Park,Scarborough Town Centre,Wexford Heights",15,15,15,15,15,15
"East Birchmount Park,Ionview,Kennedy Park",15,15,15,15,15,15


In [120]:
#Let's find out how many unique categories can be curated from all the returned venues

In [121]:
print('There are {} uniques categories.'.format(len(to_venues['Venue Category'].unique())))

There are 78 uniques categories.


Analyze Each Neighborhood

In [122]:
# one hot encoding
to_onehot = pd.get_dummies(to_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
to_onehot['Neighborhood'] = to_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [to_onehot.columns[-1]] + list(to_onehot.columns[:-1])
to_onehot = to_onehot[fixed_columns]

to_onehot.head()

,Neighborhood,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,Bakery,Bank,Beach,Beer Store,Bowling Alley,...,Spa,Sports Bar,Sri Lankan Restaurant,Supermarket,Thai Restaurant,Trail,Train Station,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Yoga Studio
0,"Rouge,Malvern",0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
1,"Rouge,Malvern",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Rouge,Malvern",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Rouge,Malvern",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"Rouge,Malvern",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [123]:
to_onehot.shape

(216, 79)

In [124]:
to_grouped = to_onehot.groupby('Neighborhood').mean().reset_index()
to_grouped

,Neighborhood,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,Bakery,Bank,Beach,Beer Store,Bowling Alley,...,Spa,Sports Bar,Sri Lankan Restaurant,Supermarket,Thai Restaurant,Trail,Train Station,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Yoga Studio
0,Agincourt,0.000000,0.000000,0.000000,0.000000,0.066667,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.066667,0.066667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,"Agincourt North,L'Amoreaux East,Milliken,Steel...",0.000000,0.000000,0.000000,0.000000,0.066667,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.066667,0.000000,0.000000
2,"Birch Cliff,Cliffside West",0.000000,0.000000,0.076923,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.076923,0.000000,0.000000,0.000000,0.000000,0.000000
3,Cedarbrae,0.000000,0.066667,0.000000,0.000000,0.066667,0.066667,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.066667,0.000000,0.000000,0.000000,0.000000,0.066667
4,"Clairlea,Golden Mile,Oakridge",0.000000,0.000000,0.000000,0.000000,0.066667,0.066667,0.000000,0.066667,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,"Clarks Corners,Sullivan,Tam O'Shanter",0.000000,0.000000,0.000000,0.000000,0.000000,0.066667,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.066667,0.000000,0.000000,0.000000,0.000000,0.000000
6,"Cliffcrest,Cliffside,Scarborough Village West",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.166667,0.000000,0.000000,...,0.000000,0.083333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,"Dorset Park,Scarborough Town Centre,Wexford He...",0.066667,0.000000,0.000000,0.000000,0.066667,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,"East Birchmount Park,Ionview,Kennedy Park",0.000000,0.000000,0.000000,0.000000,0.000000,0.066667,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,"Guildwood,Morningside,West Hill",0.000000,0.000000,0.000000,0.000000,0.000000,0.066667,0.000000,0.066667,0.000000,...,0.000000,0.066667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [125]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [126]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = to_grouped['Neighborhood']

for ind in np.arange(to_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(to_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Chinese Restaurant,Caribbean Restaurant,Cantonese Restaurant,Seafood Restaurant,Noodle House,Breakfast Spot,Restaurant,Indian Restaurant,Malay Restaurant,Sri Lankan Restaurant
1,"Agincourt North,L'Amoreaux East,Milliken,Steel...",Chinese Restaurant,Caribbean Restaurant,Event Space,Noodle House,Malay Restaurant,Park,Dessert Shop,Korean Restaurant,Pizza Place,Coffee Shop
2,"Birch Cliff,Cliffside West",Gym,Gym Pool,Auto Garage,General Entertainment,Thai Restaurant,Dessert Shop,Diner,Ice Cream Shop,Skating Rink,College Stadium
3,Cedarbrae,Indian Restaurant,Yoga Studio,Pharmacy,Athletics & Sports,Bakery,Bank,Burger Joint,Caribbean Restaurant,Chinese Restaurant,Coffee Shop
4,"Clairlea,Golden Mile,Oakridge",Coffee Shop,Bus Line,Ice Cream Shop,Fast Food Restaurant,Intersection,Bus Station,Restaurant,Metro Station,Sandwich Place,Beer Store


Cluster Neighborhoods

Run k-means to cluster the neighborhood into 3 clusters.

In [127]:
# import k-means from clustering stage
from sklearn.cluster import KMeans

# set number of clusters
kclusters = 5

to_grouped_clustering = to_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(to_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 3, 0, 1, 0, 4, 1, 1, 1], dtype=int32)

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [128]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

to_merged = toro_data

In [129]:
# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
to_merged = to_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')
to_merged.dropna
to_merged.head() # check the last columns!

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353,1.0,Fast Food Restaurant,Coffee Shop,Trail,Spa,Chinese Restaurant,Martial Arts Dojo,Paper / Office Supplies Store,Bus Station,Caribbean Restaurant,Fruit & Vegetable Store
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497,2.0,Burger Joint,Park,Italian Restaurant,Playground,Breakfast Spot,Fast Food Restaurant,Department Store,Dessert Shop,Diner,Discount Store
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711,1.0,Pizza Place,Fast Food Restaurant,Sandwich Place,Beer Store,Liquor Store,Juice Bar,Discount Store,Pharmacy,Coffee Shop,Sports Bar
3,M1G,Scarborough,Woburn,43.770992,-79.216917,1.0,Coffee Shop,Park,Indian Restaurant,Pharmacy,Mobile Phone Shop,Fast Food Restaurant,Chinese Restaurant,Fried Chicken Joint,Food & Drink Shop,Fish Market
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,0.0,Indian Restaurant,Yoga Studio,Pharmacy,Athletics & Sports,Bakery,Bank,Burger Joint,Caribbean Restaurant,Chinese Restaurant,Coffee Shop


In [130]:
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.8.1

Please update conda by running

    $ conda update -n base -c defaults conda



# All requested packages already installed.



In [131]:
    
    from bs4 import BeautifulSoup as bts # library for web scraping
    import numpy as np # library to handle data in a vectorized manner
    import pandas as pd # library for data analysis
    from pandas.io.json import json_normalize
    import matplotlib.cm as cm
    import matplotlib.colors as colors
    import requests # library to handle requests
    from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
    import matplotlib as mp # library for visualization
    from sklearn.cluster import KMeans # import k-means from clustering stage
    from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
    import folium # map rendering library
    import lxml
    import re
    from time import sleep

In [132]:
# create map
map_clusters = folium.Map(location=[latitude, longitude ], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

In [133]:
# add markers to the map
markers_colors = []

for lat, lon, poi, cluster in zip(to_merged['Latitude'], to_merged['Longitude'], to_merged['Neighborhood'], to_merged['Cluster Labels']):
    #print(type(cluster))
    #print(cluster)
    if np.isnan(cluster):
        cluster = np.nan_to_num(cluster)
           
    cluster = int(round(cluster))
    
    #print(type(cluster))
    #print(cluster)
    
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
     

In [134]:
map_clusters